In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("ps3.ipynb")

# STATS 507 
## Problem Set 3
All functions will be tested by visible as well as hidden tests. The maximum amount of time any function is allowed to run is 10 seconds.

You may use any module in the [standard library](https://docs.python.org/3/library/) to solve these problems. You may not (yet) use other modules (scipy, numpy, pandas, etc.) that require installation. Here are a few modules that may prove particularly useful:

In [2]:
import collections
import string
import itertools

### Question 1: Infinite sequences
For each of the problems below, write a function which generates the given infinite sequence. We should be able to use your generators to access any entry of the sequence no matter how deep. 

**1(a)** (2 pts) The prime numbers are

$$2, 3, 5, 7, 11, 13, 17, \dots$$

Give a generator for the primes. (Note: many algorithms exist for this problem. Yours should be efficient enough that we can use it to generate reasonably large prime numbers.)

In [25]:
def primes():
    yield 2
    primes = [2] # prime numbers list
    candidate = 3 # current candidate to check for primality

    while True:
        if all(candidate % prime != 0 for prime in primes):
            primes.append(candidate)
            yield candidate
        candidate += 2

In [26]:
grader.check("q1a")

q1a results: All test cases passed!

**1(b)** (3 pts) The *ruler sequence* is

$$1, 2, 2, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 7\dots$$

The first number (1) appears once; the next two numbers (2 and 3) appear twice, the next three numbers appear three times, etc.

Hint: You are highly encouraged to use relevant functions from itertools to make your solution simple

In [47]:
def ruler():
    counter = 1
    n = 1
    while True:
        for i in range(counter):
            for j in range(counter):
                yield n
            n += 1
        counter += 1

In [52]:
# Test
ruler_seq = ruler()
print([next(ruler_seq) for _ in range(26)])

[1, 2, 2, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 7, 8, 8, 8, 8, 9, 9, 9, 9]


In [53]:
grader.check("q1b")

q1b results: All test cases passed!

**1(c)** (3 pts) The look-and-say sequence  

$$1, 11, 21, 1211, 111221, 312211, 13112221, 1113213211, \dots$$

is generated as follows:

1. The first entry is one.
2. The second entry is generated by reading the first entry aloud: "one one"
3. The third entry is generated by reading the second entry aloud: "two ones"
4. The fourth entry is generated by reading the third entry aloud: "one two and one one"
5. The fifth entry is therefore "one one, one two, and two ones"

... and so forth. Note that each entry of the sequence should be a `str` object.


In [55]:
def look_say():
    s = "1"
    while True:
        yield s
        s = "".join(str(len(list(group))) + digit for digit, group in itertools.groupby(s))

In [56]:
grader.check("q1c")

q1c results: All test cases passed!

### Question 2: Simple ciphers

A *cipher* is an algorithm for encrypting or decrypting a text message, called *plaintext*, into *ciphertext*. In this exercise, we will assume that all plaintext to be encrypted or decrypted are composed of the lower-case Roman alphabet, without any punctuation or whitespace. Examples of such messages could be

```attackatdawn```

or 

```iamajellyfilleddonut```.

**2(a)** (4 pts) One of the oldest known ciphers is the [Caesar cipher](https://en.wikipedia.org/wiki/Caesar_cipher) or shift cipher, attributed to the Roman emperor. <img src="https://upload.wikimedia.org/wikipedia/commons/b/b4/Bust_of_Julius_Caesar_from_History_of_the_World_%281902%29.png" width=100 style="float: right; margin: 0 0 10px 10px;" /> The cipher works by shifting all letters of the alphabet by a pre-specified integer $k$. For example, if $k=+1$ then the plaintext

```thequickbrownfoxjumpsoverthelazydog```

encrypts to the ciphertext

```uifrvjdlcspxogpykvnqtpwfsuifmbazeph```

(Notice that in this example the letter `z` in `lazy` wrapped around to become an `a`.) 

Write two functions, `enc_caesar(s, k)` and `dec_caesar(s, k)` that respectively encrypt and decrypt the string `s` using the Caesar cipher, based on the integer key `k`.

Note: Functions here are defined with annotations for input arguments and return type. Refer: https://peps.python.org/pep-0484/

Hint: You are encouraged to look into string constants and string methods to get a simpler solution

In [59]:
def enc_caesar(s: str, k: int) -> str:
    '''
    Encrypt the message s using Ceasar cipher with key k.
    
    >>> enc_caeasar('thequickbrownfoxjumpsoverthelazydog', 1)
    'uifrvjdlcspxogpykvnqtpwfsuifmbazeph'
    '''
    s = s.lower()
    result = ""
    for c in s:
        if c in string.ascii_lowercase:
            shift_index = (string.ascii_lowercase.find(c) + k) % 26
            c = string.ascii_lowercase[shift_index]
        result += c
    return result
    
def dec_caesar(s: str, k: int) -> str:
    '''
    Decrypt the message s using Ceasar cypher with key k.
    
    >>> dec_caeasar('uifrvjdlcspxogpykvnqtpwfsuifmbazeph', 1)
    'thequickbrownfoxjumpsoverthelazydog'
    '''
    return enc_caesar(s, -k)

In [60]:
grader.check("q2a")

q2a results: All test cases passed!

<img src="https://upload.wikimedia.org/wikipedia/commons/9/9a/Vigenère_square_shading.svg" style="float: right; margin: 0 0 10px 10px" width=300 />


**2(b)** (4 pts) The Caesar ciphers are easily broken since the key can either be guessed, or is often widely available. A much more secure cipher is the [Vigenère cipher](https://en.wikipedia.org/wiki/Vigenère_cipher), which is keyed using a pre-specified word. The cipher works using a [tabula recta](https://en.wikipedia.org/wiki/Tabula_recta), a picture of which is shown to the right. 

Given a message `s` to be encrypted, the key `k` is repeated until it has the same length as `s`. Then each entry of the encrypted message `e` is obtained by looking up the corresponding **row** in `k`, and the correspending  **column** in `s`, in the tabula recta.

For example, suppose the message is `hello`, and the key is `sun`. We repeat the key until it has the same length as hello: `sunsu`. Then we encrypt the message by looking up the each entry in the table: `(s,h) (u,e) (n,l) (s,l) (u,o)`. The resulting string is `zyydi`.

In [61]:
def enc_vignere(s: str, k: str) -> str:
    "Encrypt s using Vignere cipher with key k"
    s = s.lower()
    k = k.lower()
    result = ""
    klen = len(k)
    for i in range(len(s)):
        if s[i] in string.ascii_lowercase:
            shift = string.ascii_lowercase.find(k[i % klen])
            shift_index = (string.ascii_lowercase.find(s[i]) + shift) % 26
            c = string.ascii_lowercase[shift_index]
            result += c
    return result


def dec_vignere(s: str, k: str) -> str:
    "Decrypt s which was encrypted by Vignere cipher using key k"
    s = s.lower()
    k = k.lower()
    result = ""
    klen = len(k)
    for i in range(len(s)):
        if s[i] in string.ascii_lowercase:
            shift = string.ascii_lowercase.find(k[i % klen])
            shift_index = (string.ascii_lowercase.find(s[i]) - shift + 26) % 26
            c = string.ascii_lowercase[shift_index]
            result += c
    return result

In [62]:
grader.check("q2b")

q2b results: All test cases passed!

### Question 3: Book ciphers
<img src="https://upload.wikimedia.org/wikipedia/commons/e/e8/King-James-Version-Bible-first-edition-title-page-1611.png" width=200 style="float: right; margin: 0 0 10px 10px" />

A [bible cipher](https://en.wikipedia.org/wiki/Book_cipher) uses the King James Bible (or some other widely available text) as the key. The cipher works by replacing each word in the message with a reference to a particular location in the Bible where that word occurs. 

The bible is organized hierarchically into *books*, *chapters*, and *verses*. For example, the first sentence in the King James Bible is 

>In the beginning God created the heaven and the earth. (Genesis 1:1)

This is the first sentence of the book of Genesis, chapter 1, verse 1. In a bible cipher, an occurrence the word "beginning" in the plaintext message could be replaced by the tuple `(0,0,0,2)`, in reference to the third word of the first book, first chapter, and first verse of the Bible.

**3(a)** (3 pts) The file `kjv.txt` contains complete text of the King James Bible. Each book begins with 2 hash symbols, comma separated chapter and verse numberes are grouped together with `[]`. A tab character separates the chapter, verse grouping from the line.

Parse this file into a data structure `bible` 
such that calling `bible[book][chapter][verse][i]` returns the `i`th word of the corresponding book, chapter and verse. Here `book`, `chapter`, and `verse`, and `i` are integers representing the (0-indexed) position of corresponding word. You should convert all words to lowercase and remove anything that is not an alphabetic character or a space:

```
>>> bible[0][0][0]
['in', 'the', 'beginning', 'god', 'created', 'the', 'heaven', 'and', 'the', 'earth']
>>> bible[42][2][15]
['for', 'god', 'so', 'loved', 'the', 'world', 'that', 'he', 'gave', 'his', 'only', 'begotten', 'son']
```

In [ ]:
filename = 'kjv.txt'
with open(filename, 'r') as file:
    lines = file.readlines()

# Create a list of unique books for indexing 
unique_books = []
for line in lines:
    if line.startswith("##"):  # this is a book title
        book_name = line[2:].strip()
        if book_name not in unique_books:
            unique_books.append(book_name)

# Initialize the nested dictionary
bible = { book_index: {} for book_index in range(len(unique_books)) }

# Go through each line of the file
for line in lines:
    line = line.strip()  # remove leading/trailing whitespaces
    if not line: continue  # skip empty lines

    if line.startswith("##"):  # this is a book title
        current_book = unique_books.index(line[2:].strip())

    elif line.startswith("["):  # this is a chapter:verse and text
        parts = line.split("\t")
        chapter_verse, current_text = parts[0][1:-1], parts[1] if len(parts) > 1 else ""
        chapter, verse = map(int, chapter_verse.split(":"))
        chapter -= 1
        verse -= 1

        # populate the nested dictionary
        if chapter not in bible[current_book]:
            bible[current_book][chapter] = {}

        bible[current_book][chapter][verse] = current_text.strip()


In [137]:
grader.check("q3a")

q3a results:
    q3a - 1 result:
        ❌ Test case failed
        Trying:
            assert bible[0][0][0] == ['in', 'the', 'beginning', 'god', 'created', 'the', 'heaven', 'and', 'the', 'earth']
        Expecting nothing
        **********************************************************************
        Line 1, in q3a 0
        Failed example:
            assert bible[0][0][0] == ['in', 'the', 'beginning', 'god', 'created', 'the', 'heaven', 'and', 'the', 'earth']
        Exception raised:
            Traceback (most recent call last):
              File "/opt/conda/lib/python3.11/doctest.py", line 1351, in __run
                exec(compile(example.source, filename, "single",
              File "<doctest q3a 0[0]>", line 1, in <module>
                assert bible[0][0][0] == ['in', 'the', 'beginning', 'god', 'created', 'the', 'heaven', 'and', 'the', 'earth']
                       ~~~~~^^^
            KeyError: 0
        Trying:
            assert bible[42][2][15] == ['for', 'god', 'so', 'loved', 'the', 'world', 'that', 'he', 'gave', 'his', 'only', 'begotten', 'son', 'that', 'whosoever', 'believeth', 'in', 'him', 'should', 'not', 'perish', 'but', 'have', 'everlasting', 'life']
        Expecting nothing
        **********************************************************************
        Line 2, in q3a 0
        Failed example:
            assert bible[42][2][15] == ['for', 'god', 'so', 'loved', 'the', 'world', 'that', 'he', 'gave', 'his', 'only', 'begotten', 'son', 'that', 'whosoever', 'believeth', 'in', 'him', 'should', 'not', 'perish', 'but', 'have', 'everlasting', 'life']
        Exception raised:
            Traceback (most recent call last):
              File "/opt/conda/lib/python3.11/doctest.py", line 1351, in __run
                exec(compile(example.source, filename, "single",
              File "<doctest q3a 0[1]>", line 1, in <module>
                assert bible[42][2][15] == ['for', 'god', 'so', 'loved', 'the', 'world', 'that', 'he', 'gave', 'his', 'only', 'begotten', 'son', 'that', 'whosoever', 'believeth', 'in', 'him', 'should', 'not', 'perish', 'but', 'have', 'everlasting', 'life']
                       ~~~~~^^^^
            KeyError: 42

**3(b)** (3 pts) In order to encode quickly encode a message, we need to be able to efficiently map a given word to all of its location(s) in the bible. Create a second data structure `bible_inv` such that `bible_inv[word]` contains a list of all the locations where that word occurs in `bible`. Each location should be encoded as a 4-tuple of integers, such that the following identity holds:
```
for b, c, v, w in bible_inv[word]:
    assert bible[b][c][v][w] == word
```

In [ ]:
bible_inv = ...

In [ ]:
grader.check("q3b")

**3(c)** (3 pts) Finally, create the functions `enc_bible(s, bible_inv)` and `dec_bible(s, bible)` which encode and decode the string `s` given the mappings `bible` and `bible_inv` above. The output of `enc_bible` should be a string of hyphen-separated 4-tuples:

```
>>> m = enc_bible("the eagle flies at dawn", bible_inv)
>>> m
'5-20-42-7 25-16-2-9 18-77-44-5 25-23-17-10 39-27-0-10'
>>> dec_bible(m, bible)
"the eagle flies at dawn"
```

You may assume that any plaintext phrase `s` consists of entirely of unpunctuated lowercase words that exist in the bible.

**Note**: the output of `enc_bible` is not necessarily unique.

In [ ]:
def enc_bible(s, bible_inv):
    ...
def dec_bible(s, bible):
    ...

In [ ]:
grader.check("q3c")

**3(d)** (3 pts) One weakness of a book cipher is that it is not possible to encrypt a message containing words that are not found in the book. For example, I cannot encrypt the message `"i heart sushi"` using the bible cipher. Work around this limitation by creating a function `closest_encryptable(s, bible_inv)` which returns a copy of the plaintext `s` where each word in `s` that is not found in the bible has been replaced by its "nearest match" that *is* in the bible. 

```
>>> closest_encryptable("i heart sushi", bible_inv)
"i heart susi"
```

To measure the closeness between two words, we will use the function `difflib.SequenceMatcher`:

In [ ]:
from difflib import SequenceMatcher
print([SequenceMatcher(a="sushi", b=w).ratio() for w in ("shush", "pizza")])  # high vs low score

In the event of a tie score, pick the word that comes last in alphabetical order. You can again assume that `s` is composed of unpunctuated lower-case words separated by spaces, as in the example.

In [ ]:
def closest_encryptable(s, bible_inv):
    ...

In [ ]:
grader.check("q3d")

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

Upload this .zip file to Gradescope for grading.

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False, run_tests=True)